In [159]:
# IMPORTS
 
#  Import libraries for numpy, random generator, and 
import numpy as np
import random
from collections import defaultdict, Counter

# Set the seed to 42
np.random.seed(42)
random.seed(42)

In [160]:
### HELPER: Proportion Weights (per disruptor group)

# Make a function that gets the proportion of mobilized, assisted and unprepared
# that come to our queuing system based on the data collected
def get_weights():
    probs = {
        "mobilized": (0.106383 + 0.037736 + 0.147727) / 3,
        "assisted":  (0.446809 + 0.132075 + 0.272727) / 3,
        "unprepared":(0.446809 + 0.188679 + 0.204545) / 3,
        }
# The proportion of normal
# people should then be 1 minus each disruptor weighed proportion
    total = sum(probs.values())
    probs["normal"] = 1 - total
    return list(probs.keys()), list(probs.values())

# Print each proportion for each group
def print_weights():
    groups, weights = get_weights()
    for group, weight in zip(groups, weights):
        print(f"{group}: {weight:.4f}")

# Use the function to print the weights
# print_weights()

In [161]:
### Helper: Compute Lamda (per shift)

# Compute the lambda value for the queuing system based on the total number of customers
# for each shift which is 249, 322, 446 for 8am, 12pm, and 4pm respectively
# It should be computed as (total number of people) / (shift duration in seconds)*(2 lanes)
def compute_lambda(total_customers):
    shift_duration_seconds = 30 * 60  # Shift duration in seconds (30 minutes)
    lanes = 2  # Number of lanes
    lamda = total_customers / (shift_duration_seconds * lanes)
    return lamda

# Print each lambda value for each shift to 4 decimal places
def print_lambda(total_customers):
    lamda = compute_lambda(total_customers)
    print(f"Lambda: {lamda:.4f}")

# Use the function to print the lambda value for 8am
# print_lambda(249)

In [162]:
### HELPER: Mixture Distribution Service Time (per disruptor group)

# Create a function that computes the service time for each disruptor group and
# a normal person
# It should use exponential distribution for disruptor groups with means of 7, 20, 21
# seconds for mobbilized, assisted, unprepared respectively 
# It should use uniform distribution for normal people between 1 and 2 seconds
def compute_service_time(group):
    if group == "mobilized":
        return np.random.exponential(7)
    elif group == "assisted":
        return np.random.exponential(20)
    elif group == "unprepared":
        return np.random.exponential(21)
    elif group == "normal":
        return np.random.uniform(1, 2)
    else:
        raise ValueError("Invalid group")

In [163]:
### HELPER: Shift Data (per shift)

# Create a function that stores lambda, lane probabilities, and number 
# of wrong exitors for each shift
# The lambda value should be computed using the compute_lambda function
# The lane probabilities should be extracted from the data and that is
    # 8am - 0.614, 0.386 for line 1 and line 2 respectively
    # 12pm - 0.632, 0.368 for line 1 and line 2 respectively
    # 4pm - 0.240, 0.760 for line 1 and line 2 respectively

def shift_data(shift_time):
    # Compute lambda for the shift
    if shift_time == "8am":
        lamda = compute_lambda(249)
    elif shift_time == "12pm":
        lamda = compute_lambda(322)
    elif shift_time == "4pm":
        lamda = compute_lambda(446)
    else:
        raise ValueError("Invalid shift time")

    # Save wrong exitors for each shift, it is 4 for 8am and 12 for
    # for the other shifts
    if shift_time == "8am":
        wrong_exitors = 4
    elif shift_time == "12pm":
        wrong_exitors = 12
    elif shift_time == "4pm":
        wrong_exitors = 12
    else:
        raise ValueError("Invalid shift time")

    # Define lane probabilities based on the shift time
    if shift_time == "8am":
        lane_probabilities = [0.614, 0.386]
    elif shift_time == "12pm":
        lane_probabilities = [0.632, 0.368]
    elif shift_time == "4pm":
        lane_probabilities = [0.240, 0.760]
    else:
        raise ValueError("Invalid shift time")
    
    # Return a dictionary with the computed values
    return {
        "lambda": lamda,
        "lane_probabilities": lane_probabilities,
        "wrong_exitors": wrong_exitors
    }

##### SIMULATION

In [164]:
# SIMULATION MODEL

np.random.seed(42)
random.seed(42)

# Write a function for a simulation model that takes in shift time, number of customers,
# and the duration of the shift in seconds
def simulation_model(shift_time, num_customers=500, shift_duration_seconds=30*60):
    
    ## 1. Get all the necessary data for the shift for this run of the simulation
    # We need lambda, lane probabilities, and number of wrong exitors
    shift_info = shift_data(shift_time)
    lamda = shift_info["lambda"]
    lane_probabilities = shift_info["lane_probabilities"]
    wrong_exitors = shift_info["wrong_exitors"]
    # Get all the necessary data for the disruptor groups
    groups, weights = get_weights()
    # Calculate when each wrong exitor will arrive at the queuing system
    # which should be evenly distributed across the shift duration
    wrong_exitor_arrivals = np.linspace(0, shift_duration_seconds, wrong_exitors + 1)[1:]

    ## 2. Initialize the simulation variables
    # Save 3 scanners in a boolean list for lane 1 and lane 2 to mark
    # whether they are available or not
    scanners = {1: [0, 0, 0],  # Lane 1 scanners, 3 scanners available
                2: [0, 0, 0]}  # Lane 2 scanners, 3 scanners available
    # Create 5 empty arrays for arrivals, service times, wait times, lanes, and groups
    # to store the simulation results
    arrivals = np.zeros(num_customers)
    service_times = np.zeros(num_customers)
    wait_times = np.zeros(num_customers)
    lanes = np.zeros(num_customers, dtype=int)
    groups_array = np.zeros(num_customers, dtype=object)
    # Create a variable called now to keep track of the current time in the simulation
    now = 0.0
    # Create a variable called wrong_exitor_index to keep track of the index of the next
    # wrong exitor to arrive at the queuing system
    wrong_exitor_index = 0

    ## 3. Simulate each customer
    # Start with a for loop in the range of the number of customers
    for i in range(num_customers):

        # 3a. Compute the interarrival time for the next customer using the exponential distribution
        # divided across 2 lanes
        interarrival_time = np.random.exponential(1 / (lamda * 2)) 
        # Update the current time in the simulation
        now += interarrival_time

        # 3b. Randomly assign the lane and group for the customer
        lane = np.random.choice([1, 2], p=lane_probabilities)
        group = np.random.choice(groups, p=weights)

        # 3c. Draw the service time for the customer using the compute_service_time function
        service_time = compute_service_time(group)

        # 3d. Check if the scanner for the assigned lane is available
        # Keep a variable called scanner_times to keep track of the time when each scanner will be available
        scanner_times = scanners[lane]
        # Enumerate all the scanners in the lane that are available
        available = [i for i, t in enumerate(scanner_times) if t <= now]
        # If a scanner is available, use it with zero wait
        # else if no scanners are available, wait until the soonest scanner is available
        if available:
            # Use the first available scanner
            scanner_index = available[0]
            wait_time = 0.0
            # Save a variable called start to mark when the scanner will start serving this person
            start = now
        else:
            # No scanners are available, find the soonest available scanner
            scanner_index = int(np.argmin(scanner_times))
            wait_time = max(0.0, scanner_times[scanner_index] - now)
            # Save a variable called start to mark when the scanner will start serving this person
            start = scanner_times[scanner_index]
        
        # 3e. Apply the wrong exitor case
        # Every wrong exitor adds 3 seconds and we assume they only go
        # to lane 2
        if wrong_exitor_index < len(wrong_exitor_arrivals) and now >= wrong_exitor_arrivals[wrong_exitor_index]:
            if lane == 2:
                for j in range(3):
                    if scanners[lane][j] > now:
                        scanners[lane][j] += 3
            wrong_exitor_index += 1

        # 3f. Update the scanner availability times  
        # Mark when the scanner will be free after serving this person
        end = start + service_time
        scanners[lane][scanner_index] = end

        # 3g. Store the results in the arrays
        arrivals[i] = now
        service_times[i] = service_time
        wait_times[i] = wait_time
        lanes[i] = lane
        groups_array[i] = group

    # 4. Return the results as a dictionary
    return {
        "arrivals": arrivals,
        "service_times": service_times,
        "wait_times": wait_times,
        "lanes": lanes,
        "groups": groups_array
    }



In [165]:
# RUN SIMULATION and COMPUTE RESULTS

np.random.seed(42)
random.seed(42)

# For every shift, print the shift number and run the simulation model
# We want the results to be stored in a pandas DataFrame and then compute the average wait time
# for each group and for every lane
import pandas as pd
def run_simulation_and_compute_results():
    for shift in ["8am", "12pm", "4pm"]:
        print(f"\n--- Shift: {shift} ---")
        results = simulation_model(shift)
        df = pd.DataFrame(results)

        # Group by lane
        df["lanes"] = df["lanes"].astype(int)
        avg_by_lane = df.groupby("lanes")["wait_times"].mean().to_dict()
        print("\nAverage wait time by lane:")
        for lane, avg in avg_by_lane.items():
            print(f"     lane {lane}: {avg:.2f} sec")

        overall_avg = df["wait_times"].mean()
        print(f"\nOverall average wait time: {overall_avg:.2f} sec")

# Call the function to run the simulation and compute results
run_simulation_and_compute_results()

        


--- Shift: 8am ---

Average wait time by lane:
     lane 1: 0.89 sec
     lane 2: 0.40 sec

Overall average wait time: 0.71 sec

--- Shift: 12pm ---

Average wait time by lane:
     lane 1: 1.91 sec
     lane 2: 0.11 sec

Overall average wait time: 1.26 sec

--- Shift: 4pm ---

Average wait time by lane:
     lane 1: 0.85 sec
     lane 2: 19.78 sec

Overall average wait time: 15.58 sec


##### ROBUSTNESS

In [166]:
import numpy as np
import random
import pandas as pd
import scipy.stats as stats

np.random.seed(42)
random.seed(42)

# Helper functions

# Define a get weight helper function to get the proportion of mobilized, assisted, unprepared and normal
def get_weights():
    probs = {
        "mobilized": (0.106383 + 0.037736 + 0.147727) / 3,
        "assisted":  (0.446809 + 0.132075 + 0.272727) / 3,
        "unprepared":(0.446809 + 0.188679 + 0.204545) / 3,
    }
    total = sum(probs.values())
    probs["normal"] = 1 - total
    return list(probs.keys()), list(probs.values())

# Define a lambda function that computes the lambda value for the queuing system
def compute_lambda(total_customers):
    return total_customers / (30 * 60 * 2)

# Define a function that computes the same shift data as before
def shift_data(shift_time):
    if shift_time == "8am":
        lamda = compute_lambda(249)
        lane_probs = [0.614, 0.386]
        wrong_exitors = 4
    elif shift_time == "12pm":
        lamda = compute_lambda(322)
        lane_probs = [0.632, 0.368]
        wrong_exitors = 12
    elif shift_time == "4pm":
        lamda = compute_lambda(446)
        lane_probs = [0.240, 0.760]
        wrong_exitors = 12
    else:
        raise ValueError("Invalid shift time")
    return {
        "lambda": lamda,
        "lane_probabilities": lane_probs,
        "wrong_exitors": wrong_exitors
    }

# Define a function that computes service time based on the 
# same distribution graphs but offset the mean by an additional 1 second
def compute_service_time_with_offset(group, offset=1.0):
    if group == "normal":
        return np.random.uniform(1.0, 2.0)
    else:
        base_mean = {"mobilized": 7, "assisted": 20, "unprepared": 21}[group]
        return max(np.random.exponential(base_mean + offset), 1)
    
# Simulation model

# Now define a simulation function that calculates the wait time again,
# but this time with the offset service time
def simulation_service_offset(shift_time, offset=1.0, num_customers=500, shift_duration_seconds=1800):
    shift_info = shift_data(shift_time)
    lamda = shift_info["lambda"]
    lane_probs = shift_info["lane_probabilities"]
    wrong_exitors = shift_info["wrong_exitors"]
    wrong_exitor_arrivals = np.linspace(0, shift_duration_seconds, wrong_exitors + 1)[1:]
    groups, weights = get_weights()

    scanners = {1: [0, 0, 0], 2: [0, 0, 0]}
    arrivals = np.zeros(num_customers)
    service_times = np.zeros(num_customers)
    wait_times = np.zeros(num_customers)
    lanes = np.zeros(num_customers, dtype=int)
    groups_array = np.zeros(num_customers, dtype=object)
    now = 0.0
    wrong_exitor_index = 0

    for i in range(num_customers):
        interarrival = np.random.exponential(1 / (lamda * 2))
        now += interarrival
        lane = np.random.choice([1, 2], p=lane_probs)
        group = np.random.choice(groups, p=weights)

        service_time = compute_service_time_with_offset(group, offset=offset)

        scanner_times = scanners[lane]
        available = [j for j, t in enumerate(scanner_times) if t <= now]

        if available:
            scanner_idx = available[0]
            wait_time = 0.0
            start = now
        else:
            scanner_idx = int(np.argmin(scanner_times))
            wait_time = max(0.0, scanner_times[scanner_idx] - now)
            start = scanner_times[scanner_idx]

        while wrong_exitor_index < len(wrong_exitor_arrivals) and now >= wrong_exitor_arrivals[wrong_exitor_index]:
            if lane == 2:
                for j in range(3):
                    if scanners[lane][j] > now:
                        scanners[lane][j] += 3
            wrong_exitor_index += 1

        scanners[lane][scanner_idx] = start + service_time

        arrivals[i] = now
        service_times[i] = service_time
        wait_times[i] = wait_time
        lanes[i] = lane
        groups_array[i] = group

    return pd.DataFrame({
        "arrival": arrivals,
        "service_time": service_times,
        "wait_time": wait_times,
        "lane": lanes,
        "group": groups_array
    })

# Results

# Define a function that computes the confidence interval
def confidence_interval(data, confidence=0.95):
    data = np.array(data)
    mean = np.mean(data)
    sem = stats.sem(data)
    margin = sem * stats.t.ppf((1 + confidence) / 2, df=len(data) - 1)
    return mean, mean - margin, mean + margin

# Run 100 simulations per shift with +1s error in mean 
# (the offset variable should equal 1)
for shift in ["8am", "12pm", "4pm"]:
    print(f"\n--- Shift: {shift} (Service time +1s) ---")
    wait_avgs = []
    for _ in range(1000):
        df = simulation_service_offset(shift, offset=1.0)
        wait_avgs.append(df["wait_time"].mean())
    mean, low, high = confidence_interval(wait_avgs)
    print(f"Mean wait time: {mean:.2f} sec")
    print(f"95% Confidence Interval: [{low:.2f}, {high:.2f}]")


--- Shift: 8am (Service time +1s) ---
Mean wait time: 0.91 sec
95% Confidence Interval: [0.88, 0.95]

--- Shift: 12pm (Service time +1s) ---
Mean wait time: 2.19 sec
95% Confidence Interval: [2.11, 2.26]

--- Shift: 4pm (Service time +1s) ---
Mean wait time: 20.11 sec
95% Confidence Interval: [19.17, 21.06]
